In [3]:
import pandas as pd
from sqlalchemy import create_engine, text

# ================== CONFIGURATION ==================
DB_TYPE = "mysql"              
DB_DRIVER = "pymysql"          
DB_USER = "root"
DB_PASSWORD = "Birju@123"
DB_HOST = "localhost"
DB_PORT = "3306"
DB_NAME = "luxury_housing_db"
TABLE_NAME = "luxury_sales_bangalore"

CLEANED_DATA_PATH = "D:/GUVI/myenv/Luxury_Housing_Sales_Analysis_Bengaluru/data/processed/luxury_bangalore_cleaned.csv"
CHUNKSIZE = 1000  # number of rows per batch

# ================== CREATE CONNECTION ==================
# URL-encode credentials so special characters (like '@' in password) do not break the connection string
from urllib.parse import quote_plus

user_enc = quote_plus(DB_USER)
pwd_enc = quote_plus(DB_PASSWORD)

connection_url = f"{DB_TYPE}+{DB_DRIVER}://{user_enc}:{pwd_enc}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_url)

# ================== LOAD CLEANED DATA ==================
print("📂 Loading cleaned dataset...")
df = pd.read_csv(CLEANED_DATA_PATH)
print(f"✅ Loaded {len(df):,} records from cleaned CSV")

# ================== CREATE TABLE (if not exists) ==================
create_table_query = """
CREATE TABLE IF NOT EXISTS luxury_sales_bangalore (
    Property_ID              VARCHAR(50) PRIMARY KEY,
    Project_Name             VARCHAR(255),
    Developer_Name           VARCHAR(255),
    Micro_Market             VARCHAR(100),
    Transaction_Type         VARCHAR(50),
    Purchase_Quarter          VARCHAR(20),
    Purchase_Quarter_Canonical VARCHAR(20),
    Quarter_Number           INT,
    Configuration            VARCHAR(20),
    Possession_Status        VARCHAR(100),
    Sales_Channel            VARCHAR(100),
    Buyer_Type               VARCHAR(100),
    Buyer_Comments           TEXT,
    Buyer_Comments_Sentiment FLOAT,
    NRI_Buyer                VARCHAR(10),
    NRI_Buyer_Flag           TINYINT,
    Ticket_Price_Cr          FLOAT,
    Ticket_Price_INR         BIGINT,
    Unit_Size_Sqft           FLOAT,
    Price_per_Sqft_INR       FLOAT,
    Amenity_Score            FLOAT,
    Connectivity_Score       FLOAT,
    Locality_Infra_Score     FLOAT,
    Avg_Traffic_Time_Min     FLOAT,
    Luxury_Flag              TINYINT,
    Booking_Potential_Score  FLOAT,
    created_at               TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""
with engine.connect() as conn:
    conn.execute(text(create_table_query))
    conn.commit()
    print("🛠️  Table checked/created successfully.")

# ================== INSERT DATA INTO SQL ==================
print("🚀 Inserting data into SQL...")

df.to_sql(
    name=TABLE_NAME,
    con=engine,
    if_exists="replace",    # options: 'append' or 'replace'
    index=False,
    chunksize=CHUNKSIZE,
    method="multi"
)

print("✅ Data inserted successfully!")

# ================== VALIDATE ROW COUNT ==================
with engine.connect() as conn:
    result = conn.execute(text(f"SELECT COUNT(*) FROM {TABLE_NAME};"))
    total_rows = result.scalar()
    print(f"📊 Total rows in table '{TABLE_NAME}': {total_rows:,}")


📂 Loading cleaned dataset...
✅ Loaded 101,000 records from cleaned CSV
🛠️  Table checked/created successfully.
🚀 Inserting data into SQL...
✅ Data inserted successfully!
📊 Total rows in table 'luxury_sales_bangalore': 101,000
